## Feature Engineering
Unter https://www.openml.org/d/41214 und https://www.openml.org/d/41215 finden Sie zwei Daten-
sätze eines französischen Automobilversicherers. Diese beinhalten Risikomerkmale und Schaden-
informationen zu Kraftfahrt-Haftpflicht-Versicherungsverträgen (eine Datensatzbeschreibung finden Sie
am Ende dieses Textes). Ihre Aufgabe besteht in der Modellierung der zu erwartenden Schadenhöhe
pro Versicherungsnehmer und Jahr anhand der Risikomerkmale der Kunden. Dieser Wert ist Basis für
die Berechnung eines fairen Versicherungsbeitrags.  

Die abhängige Variable ist definiert als ClaimAmount / Exposure.

### freMTPL2freq
The dataset freMTPL2freq contains risk features for 677,991 motor third-part liability policies (observed mostly on one year). See https://github.com/dutangc/CASdatasets for more details. The dataset is associated with 'Computational Actuarial Science with R' edited by Arthur Charpentier, CRC, 2018.  
- **IDpol**: ID des Vertrags
- **ClaimNb**: Anzahl Schäden im Versicherungszeitraum
- **Exposure**: Länge des Versicherungszeitraums (in Jahren) [Komponente der abhängigen Variable]
- **Area**: Area-Code des Versicherungsnehmers [unabhängige Variable]
- **VehPower**: Leistung des versicherten Kfz [unabhängige Variable]
- **VehAge**: Alter des versicherten Kfz [unabhängige Variable]
- **DrivAge**: Alter des Versicherungsnehmers [unabhängige Variable]
- **BonusMalus**: Schadenfreiheitsrabatt (französische Entsprechung der Schadenfreiheitsklasse) [unabhängige Variable]
- **VehBrand**: Marke des versicherten Kfz [unabhängige Variable]
- **VehGas**: Antrieb des versicherten Kfz [unabhängige Variable]
- **Density**: Anzahl der Einwohner pro km2 im Wohnort des Versicherungsnehmers [unabhängige Variable]
- **Region**: Region des Versicherungsnehmers [unabhängige Variable]

### freMTPL2sev
The dataset freMTPL2sev contains claim amounts for 26,639 motor third-part liability policies.
- **IDpol**: ID des Vertrags
- **ClaimAmount**: Höhe der einzelnen Schadenaufwände (mehrere Einträge pro Vertrag, falls im Zeitraum mehrere Schäden vorhanden waren.) [Komponente der abhängigen Variable]

In [1]:
import pandas as pd
import arff
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import gaussian_kde
import numpy as np

In [2]:
data_freq = arff.load('data/freMTPL2freq.arff')
data_sev = arff.load('data/freMTPL2sev.arff')

In [3]:
df_freq = pd.DataFrame(data_freq, columns=["IDpol", "ClaimNb", "Exposure", "Area", "VehPower",
"VehAge","DrivAge", "BonusMalus", "VehBrand", "VehGas", "Density", "Region"])
df_sev = pd.DataFrame(data_sev, columns=["IDpol", "ClaimAmount"])

## Todo

"Ihre Aufgabe besteht in der Modellierung der zu erwartenden Schadenhöhe pro Versicherungsnehmer und Jahr anhand der Risikomerkmale der Kunden. Dieser Wert ist Basis für die Berechnung eines fairen Versicherungsbeitrags."
--> GT ist claim_amount gefiltert nach jahr (aber: kann man aufgrund fehlender timestamps der unfälle nicht, also nehm ich gesamt summierten als GT)

1) Mapping from string values to numerical tokens
2) merging summed up claim amount grouped by id into training df
3) Normalize claim amount by exposure
4) Normalizing every column (besides prev string ones) on their max values
5) transform to numpy array
6) slice claim amount into seperate array --> X and y
7) create weights from y to make dataset unbiased
8) save X and y

### 1) mapping

In [4]:
df_freq

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1.0,0.10000,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
1,3.0,1.0,0.77000,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
2,5.0,1.0,0.75000,'B',6.0,2.0,52.0,50.0,'B12',Diesel,54.0,'R22'
3,10.0,1.0,0.09000,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'
4,11.0,1.0,0.84000,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'
...,...,...,...,...,...,...,...,...,...,...,...,...
678008,6114326.0,0.0,0.00274,'E',4.0,0.0,54.0,50.0,'B12',Regular,3317.0,'R93'
678009,6114327.0,0.0,0.00274,'E',4.0,0.0,41.0,95.0,'B12',Regular,9850.0,'R11'
678010,6114328.0,0.0,0.00274,'D',6.0,2.0,45.0,50.0,'B12',Diesel,1323.0,'R82'
678011,6114329.0,0.0,0.00274,'B',4.0,0.0,60.0,50.0,'B12',Regular,95.0,'R26'


In [5]:
print(df_freq["Area"].unique())
area_map = {}
for idx, item in enumerate(df_freq["Area"].unique()):
    area_map[item] = idx
print(area_map)
df_freq["Area"].replace(area_map, inplace=True)

["'D'" "'B'" "'E'" "'C'" "'F'" "'A'"]
{"'D'": 0, "'B'": 1, "'E'": 2, "'C'": 3, "'F'": 4, "'A'": 5}


In [6]:
print(df_freq["VehBrand"].unique())
brand_map = {}
for idx, item in enumerate(df_freq["VehBrand"].unique()):
    brand_map[item] = idx
print(brand_map)
df_freq["VehBrand"].replace(brand_map, inplace=True)

["'B12'" "'B6'" "'B3'" "'B2'" "'B5'" "'B10'" "'B14'" "'B13'" "'B4'" "'B1'"
 "'B11'"]
{"'B12'": 0, "'B6'": 1, "'B3'": 2, "'B2'": 3, "'B5'": 4, "'B10'": 5, "'B14'": 6, "'B13'": 7, "'B4'": 8, "'B1'": 9, "'B11'": 10}


In [7]:
print(df_freq["VehGas"].unique())
gas_map = {}
for idx, item in enumerate(df_freq["VehGas"].unique()):
    gas_map[item] = idx
print(gas_map)
df_freq["VehGas"].replace(gas_map, inplace=True)

['Regular' 'Diesel']
{'Regular': 0, 'Diesel': 1}


In [8]:
print(df_freq["Region"].unique())
reg_map = {}
for idx, item in enumerate(df_freq["Region"].unique()):
    reg_map[item] = idx
print(reg_map)
df_freq["Region"].replace(reg_map, inplace=True)

["'R82'" "'R22'" "'R72'" "'R31'" "'R91'" "'R52'" "'R93'" "'R11'" "'R24'"
 "'R94'" "'R83'" "'R54'" "'R26'" "'R53'" "'R73'" "'R42'" "'R25'" "'R21'"
 "'R41'" "'R43'" "'R74'" "'R23'"]
{"'R82'": 0, "'R22'": 1, "'R72'": 2, "'R31'": 3, "'R91'": 4, "'R52'": 5, "'R93'": 6, "'R11'": 7, "'R24'": 8, "'R94'": 9, "'R83'": 10, "'R54'": 11, "'R26'": 12, "'R53'": 13, "'R73'": 14, "'R42'": 15, "'R25'": 16, "'R21'": 17, "'R41'": 18, "'R43'": 19, "'R74'": 20, "'R23'": 21}


In [9]:
df_freq

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1.0,0.10000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0
1,3.0,1.0,0.77000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0
2,5.0,1.0,0.75000,1,6.0,2.0,52.0,50.0,0,1,54.0,1
3,10.0,1.0,0.09000,1,7.0,0.0,46.0,50.0,0,1,76.0,2
4,11.0,1.0,0.84000,1,7.0,0.0,46.0,50.0,0,1,76.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
678008,6114326.0,0.0,0.00274,2,4.0,0.0,54.0,50.0,0,0,3317.0,6
678009,6114327.0,0.0,0.00274,2,4.0,0.0,41.0,95.0,0,0,9850.0,7
678010,6114328.0,0.0,0.00274,0,6.0,2.0,45.0,50.0,0,1,1323.0,0
678011,6114329.0,0.0,0.00274,1,4.0,0.0,60.0,50.0,0,0,95.0,12


### 2) merging claim amount with df

In [10]:
df_freq

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1.0,0.10000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0
1,3.0,1.0,0.77000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0
2,5.0,1.0,0.75000,1,6.0,2.0,52.0,50.0,0,1,54.0,1
3,10.0,1.0,0.09000,1,7.0,0.0,46.0,50.0,0,1,76.0,2
4,11.0,1.0,0.84000,1,7.0,0.0,46.0,50.0,0,1,76.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
678008,6114326.0,0.0,0.00274,2,4.0,0.0,54.0,50.0,0,0,3317.0,6
678009,6114327.0,0.0,0.00274,2,4.0,0.0,41.0,95.0,0,0,9850.0,7
678010,6114328.0,0.0,0.00274,0,6.0,2.0,45.0,50.0,0,1,1323.0,0
678011,6114329.0,0.0,0.00274,1,4.0,0.0,60.0,50.0,0,0,95.0,12


In [11]:
df_sev

,IDpol,ClaimAmount
0,1552.0,995.20
1,1010996.0,1128.12
2,4024277.0,1851.11
3,4007252.0,1204.00
4,4046424.0,1204.00
...,...,...
26634,3254353.0,1200.00
26635,3254353.0,1800.00
26636,3254353.0,1000.00
26637,2222064.0,767.55


In [12]:
merged_df = pd.merge(df_freq, df_sev, on='IDpol', how='left')

In [13]:
merged_df

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
0,1.0,1.0,0.10000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0,NaN
1,3.0,1.0,0.77000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0,NaN
2,5.0,1.0,0.75000,1,6.0,2.0,52.0,50.0,0,1,54.0,1,NaN
3,10.0,1.0,0.09000,1,7.0,0.0,46.0,50.0,0,1,76.0,2,NaN
4,11.0,1.0,0.84000,1,7.0,0.0,46.0,50.0,0,1,76.0,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
679508,6114326.0,0.0,0.00274,2,4.0,0.0,54.0,50.0,0,0,3317.0,6,NaN
679509,6114327.0,0.0,0.00274,2,4.0,0.0,41.0,95.0,0,0,9850.0,7,NaN
679510,6114328.0,0.0,0.00274,0,6.0,2.0,45.0,50.0,0,1,1323.0,0,NaN
679511,6114329.0,0.0,0.00274,1,4.0,0.0,60.0,50.0,0,0,95.0,12,NaN


In [14]:
grouped_df = merged_df.groupby('IDpol')['ClaimAmount'].sum().reset_index()

In [15]:
df_freq['totalClaimAmount'] = grouped_df['ClaimAmount']

In [16]:
df_freq

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,totalClaimAmount
0,1.0,1.0,0.10000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0,0.0
1,3.0,1.0,0.77000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0,0.0
2,5.0,1.0,0.75000,1,6.0,2.0,52.0,50.0,0,1,54.0,1,0.0
3,10.0,1.0,0.09000,1,7.0,0.0,46.0,50.0,0,1,76.0,2,0.0
4,11.0,1.0,0.84000,1,7.0,0.0,46.0,50.0,0,1,76.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
678008,6114326.0,0.0,0.00274,2,4.0,0.0,54.0,50.0,0,0,3317.0,6,0.0
678009,6114327.0,0.0,0.00274,2,4.0,0.0,41.0,95.0,0,0,9850.0,7,0.0
678010,6114328.0,0.0,0.00274,0,6.0,2.0,45.0,50.0,0,1,1323.0,0,0.0
678011,6114329.0,0.0,0.00274,1,4.0,0.0,60.0,50.0,0,0,95.0,12,0.0


### 3) normalizing claims by exposure

In [17]:
df_freq['totalClaimAmountNormalized'] = df_freq['totalClaimAmount'] / df_freq['Exposure']

In [18]:
df_freq

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,totalClaimAmount,totalClaimAmountNormalized
0,1.0,1.0,0.10000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0,0.0,0.0
1,3.0,1.0,0.77000,0,5.0,0.0,55.0,50.0,0,0,1217.0,0,0.0,0.0
2,5.0,1.0,0.75000,1,6.0,2.0,52.0,50.0,0,1,54.0,1,0.0,0.0
3,10.0,1.0,0.09000,1,7.0,0.0,46.0,50.0,0,1,76.0,2,0.0,0.0
4,11.0,1.0,0.84000,1,7.0,0.0,46.0,50.0,0,1,76.0,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678008,6114326.0,0.0,0.00274,2,4.0,0.0,54.0,50.0,0,0,3317.0,6,0.0,0.0
678009,6114327.0,0.0,0.00274,2,4.0,0.0,41.0,95.0,0,0,9850.0,7,0.0,0.0
678010,6114328.0,0.0,0.00274,0,6.0,2.0,45.0,50.0,0,1,1323.0,0,0.0,0.0
678011,6114329.0,0.0,0.00274,1,4.0,0.0,60.0,50.0,0,0,95.0,12,0.0,0.0


### 4) normalizing by max

In [19]:
df_freq = df_freq.drop(columns=['IDpol', 'totalClaimAmount'])

In [20]:
scaler = MinMaxScaler()
normalized_df = pd.DataFrame(scaler.fit_transform(df_freq), columns=df_freq.columns)
print("")

In [21]:
scaler.data_min_, scaler.data_max_

(array([0.00000000e+00, 2.73224044e-03, 0.00000000e+00, 4.00000000e+00,
        0.00000000e+00, 1.80000000e+01, 5.00000000e+01, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 array([1.6000000e+01, 2.0100000e+00, 5.0000000e+00, 1.5000000e+01,
        1.0000000e+02, 1.0000000e+02, 2.3000000e+02, 1.0000000e+01,
        1.0000000e+00, 2.7000000e+04, 2.1000000e+01, 1.8524548e+07]))

In [22]:
normalized_df

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,totalClaimAmountNormalized
0,0.0625,0.048458,0.0,0.090909,0.00,0.451220,0.000000,0.0,0.0,0.045039,0.000000,0.0
1,0.0625,0.382245,0.0,0.090909,0.00,0.451220,0.000000,0.0,0.0,0.045039,0.000000,0.0
2,0.0625,0.372281,0.2,0.181818,0.02,0.414634,0.000000,0.0,1.0,0.001963,0.047619,0.0
3,0.0625,0.043476,0.2,0.272727,0.00,0.341463,0.000000,0.0,1.0,0.002778,0.095238,0.0
4,0.0625,0.417118,0.2,0.272727,0.00,0.341463,0.000000,0.0,1.0,0.002778,0.095238,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
678008,0.0000,0.000004,0.4,0.000000,0.00,0.439024,0.000000,0.0,0.0,0.122819,0.285714,0.0
678009,0.0000,0.000004,0.4,0.000000,0.00,0.280488,0.250000,0.0,0.0,0.364791,0.333333,0.0
678010,0.0000,0.000004,0.0,0.181818,0.02,0.329268,0.000000,0.0,1.0,0.048965,0.000000,0.0
678011,0.0000,0.000004,0.2,0.000000,0.00,0.512195,0.000000,0.0,0.0,0.003482,0.571429,0.0


### 5) transform to numpy array

In [23]:
data = normalized_df.to_numpy()

In [24]:
data.shape

(678013, 12)

### 6) split into X and y

In [25]:
X, y = data[:, :-1], data[:, -1]

In [26]:
X.shape, y.shape

((678013, 11), (678013,))

### 7) create weights to make dataset unbiased

In [30]:
downsampled_indices = np.random.choice(np.arange(len(y)), size=10_000, replace=False)
y_downsampled = y[downsampled_indices]

In [31]:
kde = gaussian_kde(y_downsampled)

In [32]:
pdf_values = kde.evaluate(y)

In [35]:
weights = 1.0 / (pdf_values + 1e-8)

In [36]:
weights.shape

(678013,)

### 8) save X, y and w

In [37]:
np.save("data/X.npy", X)
np.save("data/y.npy", y)
np.save("data/w.npy", weights)